In [11]:
import pandas as pd
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words("english")

In [12]:
df = pd.read_json("testdata.json")

In [13]:
df.head()

,manufacturer,name,subtitle,size,sizeuom,reviews,price,images,relatedMain,description,...,ingredients,relatedOther,rating,site,dateCreated,status,sysState,category,type,product_url
0,SkinCeuticals,C E Ferulic,A vitamin C serum that provides environmental ...,1,fl oz.,2698,166.0,[https://media.dermstore.com/catalog/100113/80...,NaN,SkinCeuticals' C E Ferulic features a synergis...,...,"Aqua/Water/Eau, Ethoxydiglycol, Ascorbic Acid,...",NaN,5 / 5,www.dermstore.com,2020-09-26T11:59:52.430Z,NEW,OPEN,Skin Care,Skin Care Treatments,https://www.dermstore.com/product_C+E+Ferulic_...
1,SkinCeuticals,Triple Lipid Restore 2:4:2,An anti-aging facial treatment with essential ...,1.6,fl oz.,770,128.0,[https://media.dermstore.com/catalog/100113/80...,NaN,Replenish your nutrients and bring your skin b...,...,"Aqua/Water/Eau, Dimethicone, Hydrogenated Poly...",NaN,5 / 5,www.dermstore.com,2020-09-26T11:59:59.090Z,NEW,OPEN,Skin Care,Skin Care Treatments,https://www.dermstore.com/product_Triple+Lipid...
2,EltaMD,UV Clear Broad-Spectrum SPF 46,"A lightweight, oil-free face sunscreen to prot...",1.7,oz,3100,36.0,[https://media.dermstore.com/catalog/500300/80...,NaN,UV Clear Broad-Spectrum SPF 46 by EltaMD provi...,...,"Active: Zinc Oxide 9.0%, Octinoxate 7.5%. Inac...",NaN,5 / 5,www.dermstore.com,2020-09-26T12:00:05.082Z,NEW,OPEN,Skin Care,Face Sun Care,https://www.dermstore.com/product_UV+Clear+Bro...
3,SkinCeuticals,Hyaluronic Acid Intensifier,A powerful face serum that deeply hydrates skin.,1,fl oz.,621,100.0,[https://media.dermstore.com/catalog/100113/80...,NaN,Reveal fresher and healthier-looking skin with...,...,"Water, Cyclohexasiloxane, Glycerin, Alcohol De...",NaN,5 / 5,www.dermstore.com,2020-09-26T12:00:09.350Z,NEW,OPEN,Skin Care,Skin Care Treatments,https://www.dermstore.com/product_Hyaluronic+A...
4,SkinMedica,HA5 Rejuvenating Hydrator,A moisturizing facial serum that provides eigh...,2,oz,314,178.0,[https://media.dermstore.com/catalog/100155/80...,NaN,SkinMedica HA5 Rejuvenating Hydrator blends fi...,...,"Water/Aqua, Dimethicone, HDI/Trimethylol Hexyl...",NaN,5 / 5,www.dermstore.com,2020-09-26T12:00:13.570Z,NEW,OPEN,Skin Care,Moisturizers,https://www.dermstore.com/product_HA5+Rejuvena...


In [14]:
df['id'] = 'id' + df.index.astype(str)

### Clear text

In [15]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = " ".join([word for word in text.split() if word not in stopwords])
    return text

### Similarity function

In [16]:
def cosine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1,-1)
    vec2 = vec2.reshape(1,-1) 
    return cosine_similarity(vec1, vec2)[0][0]

In [17]:
def similarity_text(text_from_search, text_to_search):
    sentences = [text_from_search, text_to_search]
    vectorizer = CountVectorizer().fit_transform(sentences)
    vectors = vectorizer.toarray()
    csim = cosine_sim_vectors(vectors[0],vectors[1])
    
    return csim

In [35]:
def recommendation_model(data_frame, ingredients_to_find):
    clean_ingredients_to_find = clean_text(ingredients_to_find)
    probability = {}
    for i in range(len(data_frame)):
        clean_data_frame_ingredients = clean_text(data_frame.loc[i, "ingredients"])
        csim = similarity_text(clean_ingredients_to_find, clean_data_frame_ingredients)
        probability[i] = csim
       
    nearest = {}
    for p in probability:
        if probability[p] != 0:
            nearest[data_frame.loc[p,"id"]] = probability[p] 
            
            
    result = pd.DataFrame(nearest.items(), columns=['Product', 'Probability'])
    return result.nlargest(10, 'Probability')

In [33]:
result = recommendation_model(df, "Aqua Water")

In [34]:
result

,Product,Probability
725,id1157,0.816497
1828,id2947,0.816497
812,id1306,0.577350
1162,id1872,0.577350
491,id778,0.534522
900,id1458,0.500000
396,id630,0.471405
1082,id1747,0.471405
85,id134,0.447214
637,id1013,0.433013
